In [1]:
import pandas as pd
import os

In [9]:
%env WD=/Users/amandeep/Documents/wikidata-20200504
%env COUNTRY_DIR=/Users/amandeep/Github/wikidata-fuzzy-search/utils/regional_data

env: WD=/Users/amandeep/Documents/wikidata-20200504
env: COUNTRY_DIR=/Users/amandeep/Github/wikidata-fuzzy-search/utils/regional_data


In [3]:
ls -lh "$WD"

total 59394384
-rw-r--r--  1 amandeep  staff    12G May 20 16:43 wikidata_edges_20200504.tsv.bz2
-rw-r--r--@ 1 amandeep  staff    16G May 18 11:45 wikidata_edges_20200504.tsv.gz


In [7]:
!gzcat "$WD/wikidata_edges_20200504.tsv.gz" | head











gzcat: error writing to output: Broken pipe
gzcat: /Users/amandeep/Documents/wikidata-20200504/wikidata_edges_20200504.tsv.gz: uncompress failed


#### Filter out edges for countries from the Wikidata edges file

In [10]:
!time gzcat "$WD/wikidata_edges_20200504.tsv.gz" \
  | kgtk ifexists --input-keys node1 --filter-on $COUNTRY_DIR/wikidata_countries_200.tsv --filter-keys node1 --filter-mode NONE \
  > $COUNTRY_DIR/wikidata-countries-edges.tsv


real	33m17.010s
user	36m10.586s
sys	0m38.329s


#### Remove the extra columns

In [12]:
!time cat $COUNTRY_DIR/wikidata-countries-edges.tsv | kgtk remove_columns -c \
"rank,node2;magnitude,node2;unit,node2;date,node2;item,node2;lower,node2;upper,node2;latitude,node2;longitude,node2;precision,node2;calendar,node2;entity-type" \
> $COUNTRY_DIR/wikidata-countries-edges-truncated.tsv


real	0m1.713s
user	0m1.345s
sys	0m0.149s


#### Explode the column node2

In [13]:
!time cat $COUNTRY_DIR/wikidata-countries-edges-truncated.tsv | \
kgtk explode > $COUNTRY_DIR/wikidata-countries-edges-exploded.tsv

list index out of range

real	0m6.585s
user	0m6.313s
sys	0m0.149s


#### Filter out labels and aliases for the country Qnodes

In [14]:
!time gzcat "$WD/wikidata_nodes_20200504_clean.tsv.gz" \
  | kgtk ifexists --input-keys id --filter-on $COUNTRY_DIR/wikidata_countries_200.tsv --filter-keys node1 --filter-mode NONE \
  > $COUNTRY_DIR/wikidata-countries-nodes-labels-aliases.tsv


real	2m26.871s
user	2m48.435s
sys	0m3.596s


#### Remove the columns `item` and `description` from the country nodes file

In [16]:
!time cat $COUNTRY_DIR/wikidata-countries-nodes-labels-aliases.tsv | \
kgtk remove_columns -c "type,description" \
> $COUNTRY_DIR/wikidata-countries-nodes-labels-aliases-truncated.tsv


real	0m0.925s
user	0m0.554s
sys	0m0.130s


#### Convert the wikidata-countries-nodes-labels-aliases-truncated.tsv file to KGTK Edge file format

In [18]:
!time cat $COUNTRY_DIR/wikidata-countries-nodes-labels-aliases-truncated.tsv | \
mlr --itsv --otsv reshape -i label,alias -o label,node2 \
> $COUNTRY_DIR/wikidata-countries-edges-labels-aliases-compacted.tsv

mlr: Header/data length mismatch (3 != 1) at file "(stdin)" line 201.

real	0m0.007s
user	0m0.003s
sys	0m0.004s


aliases in the above file contains multiple values separated by | We can expand the values into multiple rows with kgtk expand command

In [21]:
!cat $COUNTRY_DIR/wikidata-countries-edges-labels-aliases-compacted.tsv | \
kgtk expand --columns id label --mode NONE > $COUNTRY_DIR/wikidata-countries-edges-labels-aliases-id.tsv

rename the column `id` to `node1`

In [24]:
!kgtk rename_col $COUNTRY_DIR/wikidata-countries-edges-labels-aliases-id.tsv --old-columns id --new-columns node1 --mode NONE > \
$COUNTRY_DIR/wikidata-countries-edges-labels-aliases.tsv

We have to the column id, to the `wikidata-countries-edges-labels-aliases.tsv`

In [34]:
all_ids_dict = {}

def create_id(node1, label):
    id_key = '{}-{}'.format(node1, label)
    if id_key not in all_ids_dict:
        all_ids_dict[id_key] = 0
    else:
        all_ids_dict[id_key] += 1
    return '{}-{}-{}'.format(node1, label, all_ids_dict[id_key])



In [35]:
df = pd.read_csv('{}/wikidata-countries-edges-labels-aliases.tsv'.format(os.getenv('COUNTRY_DIR')), sep='\t', dtype=object)
for i, row in df.iterrows():
    df.loc[i, 'id'] = create_id(row['node1'], row['label'])

df.to_csv('{}/wikidata-countries-labels-aliases.tsv'.format(os.getenv('COUNTRY_DIR')), sep='\t', index=False)

#### Explode the country labels aliases file

In [36]:
!cat $COUNTRY_DIR/wikidata-countries-labels-aliases.tsv | \
kgtk explode > $COUNTRY_DIR/wikidata-countries-labels-aliases-exploded.tsv

#### Concatenate the Country Labels-Aliases files with other edges for countries, using `kgtk cat`

In [42]:
!kgtk cat $COUNTRY_DIR/wikidata-countries-labels-aliases-exploded.tsv \
$COUNTRY_DIR/wikidata-countries-edges-exploded.tsv >  $COUNTRY_DIR/countries-edges-exploded-unsorted.tsv

#### Sort the file by id and remove not required columns

In [45]:
!kgtk sort $COUNTRY_DIR/countries-edges-exploded-unsorted.tsv -c id | \
kgtk remove_columns -c "node2;kgtk:valid,node2;kgtk:list_len,node2;kgtk:si_units,node2;kgtk:language_suffix" \
> $COUNTRY_DIR/countries-edges-exploded.tsv

In [48]:
!head $COUNTRY_DIR/countries-edges-exploded.tsv 

node1	label	node2	id	node2;kgtk:data_type	node2;kgtk:number	node2;kgtk:low_tolerance	node2;kgtk:high_tolerance	node2;kgtk:units_node	node2;kgtk:text	node2;kgtk:language	node2;kgtk:latitude	node2;kgtk:longitude	node2;kgtk:date_and_time	node2;kgtk:precision	node2;kgtk:truth	node2;kgtk:symbol
Q1000	P1036	"2--6721"	Q1000-P1036-1	string					"2--6721"							
Q1000	P1081	+0.548	Q1000-P1081-1	number	0.548											
Q1000	P1081	+0.679	Q1000-P1081-10	number	0.679											
Q1000	P1081	+0.684	Q1000-P1081-11	number	0.684											
Q1000	P1081	+0.625	Q1000-P1081-12	number	0.625											
Q1000	P1081	+0.624	Q1000-P1081-13	number	0.624											
Q1000	P1081	+0.626	Q1000-P1081-14	number	0.626											
Q1000	P1081	+0.630	Q1000-P1081-15	number	0.63											
Q1000	P1081	+0.632	Q1000-P1081-16	number	0.632											
